# Optimal Workforce Size

## by Clarice Mottet

0. **[Part 0: Set Up](#part0)**
- **Objective**: Initialize programming environment.
- **Tasks:***
    - Set up libraries
    - Create functions to be used throughout notebook
    - Initialize run specific global variables

1. **[Part 1: Method - Thesis](#part1)**
- **Objective**: Find the optimal workforce size for region 0 and expand to other regions
- **Tasks:***
    - 


## <a id='part0'>Part 0: Set Up</a>
- **Objective**: Initialize programming environment.
- **Tasks:**
    - Set up libraries
    - Functions to be used throughout notebook
    - Initialize run specific global variables

- Set up libraries

In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time
from datetime import datetime

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_objval

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [5]:
dict_ = {}
dict_['var1'] = 1.0

print(dict_)

key = 'var1'
dict_[key] = [dict_[key]]

print(dict_)

{'var1': 1.0}
{'var1': [1.0]}


- Functions to be used throughout notebook

In [2]:
#Functions

#create the optimal workforce size "databases" by schedule, method (different folders)

def create_workforce_database(schedule, weekday_demand, weekend_demand):

    #load all shift information
    list_output = []

    for filename in os.listdir(r'../shifts'):
        if filename.find('json')>-1:
            with open(f'../shifts/{filename}', 'r') as file:
                data = json.load(file)
                df_ = pd.DataFrame(data)
                list_output.append(df_)

    df_output = pd.concat(list_output, ignore_index = True)

    #subset to where where shifts align with schedule demand and join into one dataframe
    df_uni = df_output[df_output['demand_type']==weekend_demand].copy().fillna(0)
    df_dp = df_output[df_output['demand_type']==weekday_demand].copy().fillna(0)
    keep_vars = ['city', 'demand_baseline', 'model',
        'max_n_shifts', 'outsourcing_cost_multiplier', 'regional_multiplier',
        'global_multiplier','region',
        'shifts_start']
    df_uni = df_uni[keep_vars]
    df_dp = df_dp[keep_vars]
    df_uni.rename(columns = {'shifts_start':'shifts_start_weekend'}, inplace = True)
    df_dp.rename(columns = {'shifts_start':'shifts_start_weekday'}, inplace = True)

    join_vars = ['city', 'demand_baseline', 'outsourcing_cost_multiplier', 'regional_multiplier',
        'global_multiplier', 'model',
        'max_n_shifts', 'region']

    df_out = df_uni.merge(df_dp, how = 'left', on = join_vars)
    df_out.loc[df_out['model']== 'fixed', 'max_n_shifts'] = np.NaN
    df_out.loc[df_out['model']== 'flex', 'max_n_shifts'] = np.NaN

    #merge with population information by city, region
    df_pop = pd.read_excel(f"../workforce_size/population.xlsx")
    df_out = df_out.merge(df_pop, how = 'left', on = ['city','region'])

    df_out = df_out[['city', 'demand_baseline', 'model',
        'max_n_shifts', 'outsourcing_cost_multiplier', 'regional_multiplier',
        'global_multiplier','region','population','shifts_start_weekday','shifts_start_weekend']]

    #add in variables to be filled by optimal workforce size algorithm
    df_out.sort_values(by = join_vars, inplace = True)
    df_out['workforce_size'] = 0
    start_time = time.time()
    datetime_format = datetime.fromtimestamp(start_time)
    df_out['date'] = datetime_format
    df_out['obj_val_prev'] = 0.0
    df_out['obj_val'] = 0.0
    df_out['obj_val_next'] = 0.0
    for method in ['method_region0','method_smallest_region_by_shifts','method_all_regions']:
        df_out.to_excel(f'../workforce_size/{schedule}/{method}.xlsx',index = False)



- Initialize run specific global variables

In [ ]:
#Run Parameters

schedule = 'schedule_thesis'
weekday_demand = 'doublepeak'
weekend_demand = 'uniform'

# create_workforce_database(schedule, weekday_demand, weekend_demand)

In [ ]:
#now need to program the three different methods for workforce sizing

#current region0
#smallest region and distinct shift start and end
#optimal workforce size individually

#compare the three methods
    #do they have similar optimal workforce size solutions
    #do they have simliar cost differentials
    #maybe take a larger sample of optimal workforce sizing with other cities too

In [2]:
#functions

#dictionary for checking work
def dict_append_workforce(dict_, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, workforce_size, objvalprev, objval, objvalnext, run_time, dict_run):
    dict_['instance_file_base'].append(base_file)
    dict_['city'].append(city)
    dict_['demand_baseline'].append(demand_baseline)
    dict_['model'].append(model)
    dict_['max_n_shifts'].append(max_n_shifts)
    dict_['outsourcing_cost_multiplier'].append(OC)
    dict_['regional_multiplier'].append(RM)
    dict_['global_multiplier'].append(GM)
    dict_['workforce_size_region0'].append(workforce_size)
    dict_['objective_value_prev'].append(objvalprev)
    dict_['objective_value'].append(objval)
    dict_['objective_value_next'].append(objvalnext)
    dict_['run_time'].append(run_time)
    dict_['dict_run'].append(dict_run)
    return dict_

#loads the optimal shift for the first region based on model
def load_shift(shift_file, OC, RM, GM, model, max_n_shifts = None):
    with open(shift_file, 'r') as file:
        data = json.load(file)
        df_shifts = pd.DataFrame(data)
    df_shifts = df_shifts[(df_shifts['outsourcing_cost_multiplier']==OC)&(df_shifts['regional_multiplier']==RM)&(df_shifts['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_shifts = df_shifts[df_shifts['model']==model]
    #partflex
    else:
        df_shifts = df_shifts[(df_shifts['model']==model)&(df_shifts['max_n_shifts']==max_n_shifts)]
    df_shifts.reset_index(drop = True, inplace=True)
    df_shifts = df_shifts[['region','shifts_start']]
    dict_shifts = {}
    for region in df_shifts['region'].unique().tolist():
        dict_shifts[region] = []
        dict_ = df_shifts[df_shifts['region']==region]['shifts_start'].tolist()[0]
        for shift_start in dict_.keys():
            dict_shifts[region].append(dict_[shift_start])
    return dict_shifts

#
def create_run_dictionary(base_file, OC, RM, GM):

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    #load all shifts
    dict_weekday = {}
    dict_weekend = {}
    dict_track = {}
    dict_opt = {}
    for model in ['fixed','flex','partflex']:
        if model == 'partflex':
            for max_n_shifts in range(2,5):
                dict_ = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts)
                dict_weekday[(model, max_n_shifts)] = dict_
                dict_ = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts)
                dict_weekend[(model, max_n_shifts)] = dict_
                dict_track[(model, max_n_shifts)] = []
                dict_opt[(model, max_n_shifts)] = 0
        else:
            dict_ = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts = None)
            dict_weekday[(model, None)] = dict_
            dict_ = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts = None)
            dict_weekend[(model, None)] = dict_
            dict_track[(model, None)] = []
            dict_opt[(model, None)] = 0

    #create a dictionary of distinct list of (model, max_n_shifts) that are the same

    list_models = [('fixed', None),('flex', None),('partflex',2),('partflex',3),('partflex',4)]
    list_ran = []
    for comb in list_models:
        list_ = list_models.copy()
        list_.remove(comb)
        for other_comb in list_:
            if (dict_weekday[comb]==dict_weekday[other_comb])&(dict_weekend[comb]==dict_weekend[other_comb])&(comb not in list_ran):
                dict_track[comb].append(other_comb)
                list_ran.append(other_comb)

    list_rmv = []
    for comb in dict_track.keys():
        ind_ = 0
        for comb_ in dict_track.keys():
            if comb in dict_track[comb_]:
                ind_ = 1

        if (dict_track[comb] == [])&(ind_ == 1):
            list_rmv.append(comb)
    for comb in list_rmv:
        del dict_track[comb]

    return dict_track

In [5]:
#use earlier minimum tries to reduce run time

list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

list_workforce_xlsx = []

for base_file in list_city_dt:

    base_file_xlsx = f'{base_file}_all_trials.xlsx'
    if base_file_xlsx in os.listdir(r'../workforce_size'):
        df_ = pd.read_excel(f'../workforce_size/{base_file_xlsx}')
        list_workforce_xlsx.append(df_)

df_workforce = pd.concat(list_workforce_xlsx, ignore_index = True)

df_workforce.columns = ['instance', 'city', 'DB', 'OC',
       'RM', 'GM', 'model', 'max_n_shifts',
       'workforce_size', 'wage_costs', 'objective_value',
       'objective_value_post_wage', 'workforce_size_trial']
df_workforce.loc[df_workforce['max_n_shifts'].isna(), 'max_n_shifts'] = 0

df_workforce['max_'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['workforce_size_trial'].transform('max')
df_workforce = df_workforce[df_workforce['max_']!=df_workforce['workforce_size_trial']]

df_workforce['max_'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['workforce_size_trial'].transform('max')
df_workforce = df_workforce[df_workforce['max_']==df_workforce['workforce_size_trial']]
df_workforce.drop(columns = ['max_'], inplace = True)
df_workforce.reset_index(drop = True, inplace = True)

def call_min_tries(df_workforce, base_file, OC, model, max_n_shifts):
    max_n_shifts_ = max_n_shifts
    if max_n_shifts == None:
        max_n_shifts_ = 0
    return max(df_workforce[(df_workforce['instance']==base_file)&(df_workforce['OC']==OC)&(df_workforce['model']==model)&(df_workforce['max_n_shifts']==max_n_shifts_)]['workforce_size_trial'].tolist()[0] - 5, 1)


In [6]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#remove berlin
val_rmv = []
for val in list_city_dt:
    if val.find('berlin') > -1:
        val_rmv.append(val)
for val in val_rmv:
    list_city_dt.remove(val)

# list_city_dt_subset = []

# for city_db in list_city_dt:
#     if (city_db.find(city_subset) > -1):
#         list_city_dt_subset.append(city_db)

print(list_city_dt)


['frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']


In [7]:
#Code that determines optimal workforce size

MAX_TRIES = 400

RM = 1.5
GM = 0.8
h_min = 40
h_max = 48
max_n_diff = 2

for base_file in list_city_dt:
# for base_file in ['berlin_db=0.50']:

    if f'{base_file}.json' not in os.listdir(r'../workforce_size/method_region0'):

        list_results = []

        #create inputs to run a trial workforce size
        weekday_ = f'{base_file}_dt=doublepeak.json'
        weekend_ = f'{base_file}_dt=uniform.json'

        instance_file_weekday = f"../instances/{weekday_}"
        instance_file_weekend = f"../instances/{weekend_}"

        shift_file_weekday = f"../shifts/{weekday_}"
        shift_file_weekend = f"../shifts/{weekend_}"

        #create dictionary for evaluating optimal workforce size
        workforce_out = f"../workforce_size/method_region0/{base_file}.json"
        dict_out = {
            'instance_file_base':[],
            'city':[],
            'demand_baseline':[],
            'model':[],
            'max_n_shifts':[],
            'outsourcing_cost_multiplier':[],
            'regional_multiplier':[],
            'global_multiplier':[],
            'workforce_size_region0':[],
            'objective_value_prev':[],
            'objective_value':[],
            'objective_value_next':[],
            'run_time':[],
            'dict_run':[]
        }

        city_pattern = r'(\w+)_db'
        db_pattern = r'db=(\d+\.\d+)'

        city_match = re.search(city_pattern, base_file)
        db_match = re.search(db_pattern, base_file)

        city = city_match.group(1) if city_match else None
        demand_baseline = float(db_match.group(1)) if db_match else None

        # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
        for OC in [1.2]:

            #run only over distinct shift combinations (avoiding duplicates but still keeping track)

            dict_run = create_run_dictionary(base_file, OC, RM, GM)
            print(f'dict_run: {dict_run}')

            dict_str = {}
            for key in dict_run.keys():
                key_str = f'{key[0]}-{key[1]}'
                dict_str[key_str] = []
                for val in dict_run[key]:
                    val_str = f'{val[0]}-{val[1]}'
                    dict_str[key_str].append(val_str)

            for comb in dict_run.keys():
                model = comb[0]
                max_n_shifts = comb[1]
                min_tries_prev = call_min_tries(df_workforce, base_file, OC, model, max_n_shifts)

                #find a minimal starting tries that is still feasible
                objval = np.nan
                while (np.isnan(objval)==True):
                    workforce_dict = {0:min_tries_prev, 1:1, 2:1, 3:1, 4:1}
                    start_time = time.time()
                    dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                    end_time = time.time()
                    run_time = end_time - start_time
                    objval = dict_results['objective_value'][0]
                    if (np.isnan(objval)==True):
                        min_tries_prev = max(min_tries_prev-5,1)
                    min_tries_feasible = max(min_tries_prev-2,1)

                    #condition to break the while loop if everything is infeasible
                    if min_tries_feasible == 1:
                        objval = 1
                    print(f'--city: {city}, DB: {demand_baseline}, OC: {OC}, model: {model}-{max_n_shifts}, MIN_TRIES: {MIN_TRIES}, run_time: {run_time}')

                #I want to have tried at least three trial sizes before getting to the minimum
                iter_ind = 0
                trial_reduction = 0
                while (iter_ind == 0):

                    list_trials = []
                    count_iter = 0
                    MIN_TRIES = max(min_tries_feasible - trial_reduction*5, 1)

                    objvalprev = 1e8
                    objval = 1e7

                    for trial_size in range(MIN_TRIES, MAX_TRIES):
                        workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                        if (objval < objvalprev)&(np.isnan(objval) == False):
                            objvalprev = objval
                            start_time = time.time()
                            dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                            end_time = time.time()
                            run_time = end_time - start_time
                            objval = dict_results['objective_value'][0]
                            objval = round(objval, 2)
                            print(f'city: {city}, DB: {demand_baseline}, OC: {OC}, model: {model}-{max_n_shifts}, trial_size: {trial_size}, run_time: {run_time}')
                            df_ = pd.DataFrame(dict_results)
                            df_['objval_round'] = objval
                            df_['workforce_size_trial'] = trial_size
                            df_['run_time'] = run_time
                            df_['dict_run'] = [dict_str]
                            list_trials.append(df_)
                            for subcomb in dict_run[comb]:
                                df_copy = df_.copy()
                                df_copy['model'] = subcomb[0]
                                df_copy['max_n_shifts'] = subcomb[1]
                                list_trials.append(df_copy)
                            count_iter += 1
                        else:
                            if count_iter >= 3:
                                iter_ind = 1
                            elif MIN_TRIES == 1:
                                iter_ind = 1
                            else:
                                trial_reduction += 1
                            break

                for df_ in list_trials:
                    list_results.append(df_)

                #keep track of optimal
                dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval, run_time, dict_str)
                for subcomb in dict_run[comb]:
                    model_ = subcomb[0]
                    max_n_shifts_ = subcomb[1]
                    dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model_, max_n_shifts_, trial_size-1, np.nan, objvalprev, objval, run_time, dict_str)

        with open(workforce_out, 'w') as f:
            json.dump(dict_out, f, indent=2)
        df_trials = pd.concat(list_results, ignore_index = True)
        df_trials.to_excel(f'../workforce_size/method_region0/{base_file}_all_trials.xlsx', index = False)


dict_run: {('fixed', None): [], ('flex', None): [('partflex', 3), ('partflex', 4)], ('partflex', 2): []}
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-06
city: frankfurt, DB: 0.5, OC: 1.2, model: fixed-None, trial_size: 10, run_time: 2.399602174758911
city: frankfurt, DB: 0.5, OC: 1.2, model: fixed-None, trial_size: 11, run_time: 2.702751636505127
city: frankfurt, DB: 0.5, OC: 1.2, model: fixed-None, trial_size: 12, run_time: 3.0964460372924805
city: frankfurt, DB: 0.5, OC: 1.2, model: fixed-None, trial_size: 13, run_time: 2.884225845336914
city: frankfurt, DB: 0.5, OC: 1.2, model: fixed-None, trial_size: 14, run_time: 2.960756540298462
city: frankfurt, DB: 0.5, OC: 1.2, model: fixed-None, trial_size: 15, run_time: 3.0471396446228027
city: frankfurt, DB: 0.5, OC: 1.2, model: flex-None, trial_size: 8, run_time: 2.2122373580932617
city: frankfurt, DB: 0.5, OC: 1.2, model: flex-None, trial_size: 9, run_time: 2.2971322536468506
city: frankfurt, DB: